# Quantum-Resilient Cryptography Benchmark Analysis

This notebook provides interactive analysis and visualization of benchmark results for PQC and classical algorithms.

Run `python run_benchmarks.py` first to generate `results/benchmark_results.json`, then execute the cells below.

## Algorithms Tested

### Classical Algorithms
- RSA-2048, RSA-4096 (asymmetric encryption)
- ECDSA-P256, ECDSA-P384 (digital signatures)
- ECDH-P256, ECDH-P384 (key exchange)
- AES-256 (symmetric encryption)

### PQC Algorithms (NIST Standardized)
- ML-KEM-512, ML-KEM-768, ML-KEM-1024 (key encapsulation)
- ML-DSA-44, ML-DSA-65, ML-DSA-87 (digital signatures)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json
from pathlib import Path

plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Notebook initialized successfully.")


In [ ]:
# Load benchmark results
results_path = Path('../results/benchmark_results.json')

if results_path.exists():
    with open(results_path, 'r') as f:
        results_data = json.load(f)
    
    df = pd.DataFrame(results_data)
    print(f"Loaded {len(df)} benchmark results")
    print(f"Algorithms: {sorted(df['algorithm_name'].unique())}")
    print(f"Data sizes: {sorted(df['data_size'].unique())}")
else:
    print("No benchmark results found. Run the benchmarking script first: python ../run_benchmarks.py")
    # Create sample data for demonstration
    df = pd.DataFrame({
        'algorithm_name': ['RSA-2048', 'ML-KEM-512', 'ECDSA-P256', 'ML-DSA-44'],
        'data_size': [1024, 1024, 1024, 1024],
        'mean_latency_ms': [5.2, 8.1, 3.8, 12.5],
        'throughput_ops_per_sec': [192, 123, 263, 80]
    })


In [ ]:
# Derive algorithm type

def categorize_algorithm(name: str) -> str:
    return 'PQC' if ('ML-KEM' in name or 'ML-DSA' in name) else 'Classical'

if 'algorithm_type' not in df.columns:
    df['algorithm_name'] = df['algorithm_name'].astype(str)
    df['algorithm_type'] = df['algorithm_name'].apply(categorize_algorithm)

# Summary statistics
summary = df.groupby(['algorithm_type', 'algorithm_name']).agg({
    'mean_latency_ms': ['mean', 'std', 'min', 'max'],
    'throughput_ops_per_sec': ['mean', 'std', 'min', 'max']
}).round(2)
summary


In [ ]:
# Latency comparison (interactive)
fig = px.box(df, x='algorithm_name', y='mean_latency_ms', color='algorithm_type',
             title='Latency Distribution by Algorithm')
fig.update_layout(xaxis_title='Algorithm', yaxis_title='Mean Latency (ms)')
fig.show()


In [ ]:
# Throughput vs data size (interactive)
if 'data_size' in df.columns and 'throughput_ops_per_sec' in df.columns:
    fig = px.scatter(df, x='data_size', y='throughput_ops_per_sec', color='algorithm_name',
                     size='mean_latency_ms', title='Throughput vs Data Size', log_x=True, log_y=True)
    fig.update_layout(xaxis_title='Data Size (bytes)', yaxis_title='Throughput (ops/sec)')
    fig.show()
else:
    print('Throughput columns not present in the dataset.')


In [ ]:
# PQC vs Classical comparison
comp = df.groupby('algorithm_type').agg({'mean_latency_ms': 'mean', 'throughput_ops_per_sec': 'mean'}).round(2)
comp_display = comp.copy()

if set(['PQC', 'Classical']).issubset(set(comp.index)):
    pqc_latency = comp.loc['PQC', 'mean_latency_ms']
    classical_latency = comp.loc['Classical', 'mean_latency_ms']
    overhead = ((pqc_latency - classical_latency) / classical_latency) * 100
    print(f"PQC overhead: {overhead:.1f}%")
else:
    print('Not enough data to compute PQC vs Classical overhead.')

comp_display


In [ ]:
# Security level vs performance trade-off
security_levels = {
    'RSA-2048': 112, 'RSA-4096': 128,
    'ECDSA-P256': 128, 'ECDSA-P384': 192,
    'ECDH-P256': 128, 'ECDH-P384': 192,
    'AES-256': 256,
    'ML-KEM-512': 128, 'ML-KEM-768': 192, 'ML-KEM-1024': 256,
    'ML-DSA-44': 128, 'ML-DSA-65': 192, 'ML-DSA-87': 256
}

df['security_level'] = df['algorithm_name'].map(security_levels)
fig = px.scatter(df, x='security_level', y='mean_latency_ms', color='algorithm_type', size='throughput_ops_per_sec',
                 hover_data=['algorithm_name'], title='Security Level vs Performance Trade-off')
fig.update_layout(xaxis_title='Security Level (bits)', yaxis_title='Mean Latency (ms)')
fig.show()


In [ ]:
# Simple recommendations based on current data
recs = []
fastest = df.loc[df['mean_latency_ms'].idxmin()]
recs.append(f"Fastest algorithm: {fastest['algorithm_name']} ({fastest['mean_latency_ms']:.2f} ms)")

pqc_df = df[df['algorithm_type'] == 'PQC']
if not pqc_df.empty:
    best_pqc = pqc_df.loc[pqc_df['mean_latency_ms'].idxmin()]
    recs.append(f"Best PQC algorithm observed: {best_pqc['algorithm_name']} ({best_pqc['mean_latency_ms']:.2f} ms)")

for i, r in enumerate(recs, 1):
    print(f"{i}. {r}")
